In [75]:
from sqlalchemy import func
from sqlalchemy import and_
import pandas as pd
from sqlalchemy import create_engine
from model import model
from sqlalchemy.orm import sessionmaker

In [3]:
database_file = '/assets/nin_database.db'
engine = create_engine(f'sqlite:///{database_file}')
Session = sessionmaker(bind=engine)
session = Session()

In [16]:
mt_list = session.query(model.MinorType).where(model.MinorType.majorType_id == 'T-4')

In [85]:
def get_mean_gad(scientificNameId, esc_ids):
    return session.query(func.avg(model.GadValue.valueM7Scale_id))\
        .filter(and_(model.GadValue.species_id == int(scientificNameId),\
        model.GadValue.elementarySegmentCombination_id.in_(esc_ids))).first()[0]

In [86]:
for mt in mt_list.all():
    print(mt._id)
    mtss_list = session.query(model.MinorTypeStandardSegment).where(model.MinorTypeStandardSegment.minorType_id == mt._id).all()
    esg_list = session.query(model.ElementarySegmentGroup).filter(model.ElementarySegmentGroup.standardSegment_id.in_(list(map(lambda x: x.standardSegment_id, mtss_list)))).all()
    esg_id_list = list(map(lambda x: x._id,esg_list))
    esc_list = session.query(model.ElementarySegmentCombination, func.count(model.ElementarySegmentCombination._id))\
        .filter(model.ElementarySegmentCombination.elementarySegmentGroup_id.in_(esg_id_list))\
        .group_by(model.ElementarySegmentCombination._id)\
        .all()
    esc_list = [x[0] for x in esc_list if x[1] == 3]
    esc_ids = [x._id for x in esc_list]  
    data[mt._id] = data.scientificNameId.apply(lambda x: get_mean_gad(x, esc_ids))

T4-1
T4-2
T4-3
T4-4
T4-5
T4-6
T4-7
T4-8
T4-9
T4-10
T4-11
T4-12
T4-13
T4-14
T4-15
T4-16
T4-17
T4-18
T4-19
T4-20


In [91]:
data.to_excel('mean_gad.xlsx', index=False)